In [1]:
import random

#Pennylane
import pennylane as qml
from pennylane import numpy as np

#PyTorch
import torch
import torchvision
from torchvision import datasets, transforms

In [2]:
dev = qml.device('default.qubit', wires = 6)
n_qubits = 6
n_layer_steps = 5
n_layers_to_add = 2
dim = 2**(n_qubits)

In [3]:
def set_random_gates(n_qubits: int):
    
    gate_set = [qml.RX, qml.RY, qml.RZ]
    chosen_gates = []
    for i in range(n_qubits):
        chosen_gate = random.choice(gate_set)
        chosen_gates.append(chosen_gate)
    return chosen_gates

def norm(complex_vector: np.array) -> float:
    """Returns the norm of a complex vector.
    
    Args:
       complex_vector (np.arrray of shape (dim,)):
           complex vector with an arbitrary dimension.
           
       
    Returns:
        norm (float): norm or magnitude of the complex vector.
            
    """
    
    #define the norm of a complex vector here
    norm = np.sqrt(sum([np.square(np.absolute(complex_vector[i])) for i in range(complex_vector.shape[0])]))
    
    return norm

def random_quantum_state(dim: int, amplitude_type: str = 'complex') -> np.array:
    """Creates a normalized random real or complex vector of a defined
    dimension.
    
    Args:
        dim (int): integer number specifying the dimension
            of the vector that we want to generate.
            
        amplitude_type (str): string indicating if you want to create a
            vector with 'complex' or 'real' number amplitudes.
            
    Returns:
        (np.array): normalized real or complex vector of the given
            dimension.
    """
    #generate an unnormalized complex vector of dimension = dim
    if amplitude_type == 'complex':
        Z = np.random.random(dim) + np.random.random(dim)*1j
    
    #generate an unnormalized real vector of dimension = dim
    elif amplitude_type == 'real':
        Z = np.random.random(dim)
    
    #normalize the complex vector Z
    Z /= norm(Z)
    
    return Z

def density_matrix(target_vector: np.array) -> np.array:
    """Return the density matrix of a pure state.
    
    Arguments:
        target_vector (np.array): Complex vector array.
        
    Returns:
        (np.array): Density matrix of shape (dim, dim),
            where dim is the dimension of the target_vector.
    """
    target_vector_reshape = target_vector.reshape((target_vector.shape[0],1))
    density_matrix = target_vector_reshape @ np.transpose(np.conjugate(target_vector_reshape))
    
    return density_matrix

In [4]:
target_vector = random_quantum_state(dim)
density_matrix = density_matrix(target_vector)

### Templates for Phase I

In [5]:
@qml.template
def apply_layer(gates, weights):
    
    for i in range(n_qubits):
        gates[i](weights[i], wires = i)
    
    tuples = [(i,i+1) for i in range(n_qubits-1)]

    for tup in tuples:
        qml.CZ(wires=[tup[0], tup[1]])
        
@qml.template #template for non-trainable part of the quantum circuit
def frozen_layers(frozen_layer_gates, frozen_layer_weights): 
    for i in range(len(frozen_layer_gates)):
        apply_layer(frozen_layer_gates[i], frozen_layer_weights[i])

@qml.qnode(dev)
def trainable_circuit(params):
    
    #apply frozen layers
    frozen_layers(layer_gates, layer_weights)
    
    #apply trainable layers
    new_weights = []
    for i in range(n_layers_to_add):
        new_weights.append(params[int(i*n_qubits):int((i+1)*n_qubits)])

    for i in range(n_layers_to_add):
        apply_layer(new_gates[i], new_weights[i])
        
    wirelist = [i for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix, wirelist))

In [6]:
def cost(x):
    
    return (1.0 - trainable_circuit(x))

In [7]:
layer_gates = []
layer_weights = []
n_layers = 2
for i in range(n_layers):
    layer_gates.append(set_random_gates(n_qubits))
    layer_weights.append(np.random.rand(n_qubits)*2*np.pi)
    
new_gates = [set_random_gates(n_qubits) for i in range(n_layers_to_add)]
new_weights = np.random.rand(n_qubits*n_layers_to_add)*2*np.pi

trainable_circuit(new_weights)

0.006377498723200836

In [8]:
print(trainable_circuit.draw())

 0: ──RZ(3.07)───╭C───RX(3.53)────────────╭C──────────RZ(3.486)──────────────╭C───────────RX(5.057)──────────────╭C────────────────────────────────────────╭┤ ⟨H0⟩ 
 1: ──RX(0.716)──╰Z──╭C──────────RX(1.9)──╰Z─────────╭C───────────RX(2.782)──╰Z──────────╭C───────────RX(6.234)──╰Z──────────╭C────────────────────────────├┤ ⟨H0⟩ 
 2: ──RY(0.643)──────╰Z─────────╭C─────────RZ(2.75)──╰Z──────────╭C───────────RZ(4.232)──╰Z──────────╭C───────────RX(5.383)──╰Z────────╭C──────────────────├┤ ⟨H0⟩ 
 3: ──RZ(5.63)──────────────────╰Z────────╭C──────────RX(3.07)───╰Z──────────╭C───────────RZ(6.106)──╰Z──────────╭C───────────RY(5.7)──╰Z──────────╭C──────├┤ ⟨H0⟩ 
 4: ──RX(3.98)────────────────────────────╰Z─────────╭C───────────RY(3.14)───╰Z──────────╭C───────────RZ(1.489)──╰Z──────────╭C─────────RY(5.135)──╰Z──╭C──├┤ ⟨H0⟩ 
 5: ──RX(4.87)───────────────────────────────────────╰Z───────────RX(3.61)───────────────╰Z───────────RX(5.84)───────────────╰Z─────────RY(5.605)──────╰Z──╰┤ ⟨H0⟩ 
H0 =
[[0.0180993

In [9]:
layer_gates

[[pennylane.ops.qubit.RZ,
  pennylane.ops.qubit.RX,
  pennylane.ops.qubit.RY,
  pennylane.ops.qubit.RZ,
  pennylane.ops.qubit.RX,
  pennylane.ops.qubit.RX],
 [pennylane.ops.qubit.RX,
  pennylane.ops.qubit.RX,
  pennylane.ops.qubit.RZ,
  pennylane.ops.qubit.RX,
  pennylane.ops.qubit.RY,
  pennylane.ops.qubit.RX]]

In [10]:
layer_gates = []
layer_weights = []
cost_list = []
for step in range(n_layer_steps):
    
    new_gates = [set_random_gates(n_qubits) for i in range(n_layers_to_add)]
    init_params = np.zeros(n_qubits*n_layers_to_add)
    
    opt = qml.GradientDescentOptimizer(stepsize = 0.4)

    opt_steps = 100

    params = init_params
    
    for i in range(opt_steps):

        params = opt.step(cost, params)
    
    
    print(step)
    print(layer_weights)
    print(cost(params))
    cost_list.append(cost(params))
    
    trained_weights = [params[int(i*n_qubits):int((i+1)*n_qubits)] for i in range(n_layers_to_add)]
    
    layer_gates += new_gates
    layer_weights += trained_weights

0
[]
0.8796879689103688
1
[array([ 2.10400159e-01,  1.66533454e-17, -5.47083698e-01,  2.63677968e-17,
        5.68989300e-17,  1.80411242e-17]), array([ 0.43871574,  0.72450691, -0.5470837 , -0.80994176,  0.45738643,
        1.15241499])]
0.7069790986837676
2
[array([ 2.10400159e-01,  1.66533454e-17, -5.47083698e-01,  2.63677968e-17,
        5.68989300e-17,  1.80411242e-17]), array([ 0.43871574,  0.72450691, -0.5470837 , -0.80994176,  0.45738643,
        1.15241499]), array([-0.45456927,  0.50659424,  1.24572831,  1.38817122,  0.30235989,
        0.5777338 ]), array([ 0.04519363,  0.02070302,  0.50438761, -0.23495578, -0.04768922,
       -0.10360508])]
0.5871134741946022
3
[array([ 2.10400159e-01,  1.66533454e-17, -5.47083698e-01,  2.63677968e-17,
        5.68989300e-17,  1.80411242e-17]), array([ 0.43871574,  0.72450691, -0.5470837 , -0.80994176,  0.45738643,
        1.15241499]), array([-0.45456927,  0.50659424,  1.24572831,  1.38817122,  0.30235989,
        0.5777338 ]), array([ 0.0

In [11]:
layer_weights

[array([ 2.10400159e-01,  1.66533454e-17, -5.47083698e-01,  2.63677968e-17,
         5.68989300e-17,  1.80411242e-17]),
 array([ 0.43871574,  0.72450691, -0.5470837 , -0.80994176,  0.45738643,
         1.15241499]),
 array([-0.45456927,  0.50659424,  1.24572831,  1.38817122,  0.30235989,
         0.5777338 ]),
 array([ 0.04519363,  0.02070302,  0.50438761, -0.23495578, -0.04768922,
        -0.10360508]),
 array([-0.46459097,  0.63507508, -0.1146228 , -0.30094421,  0.93010568,
         0.15200487]),
 array([ 0.14479619,  0.02267602, -0.1146228 , -0.71001688,  0.22014436,
        -0.07767378]),
 array([ 0.71445844,  0.75289405, -0.87384212, -0.00109433,  0.20517063,
        -0.18387147]),
 array([-0.70863476,  0.06306516, -0.27223206, -0.1096503 ,  0.06962437,
        -0.05013378]),
 array([ 0.23179883, -0.01797858,  0.29046467, -0.18620561, -0.0203647 ,
        -0.01099372]),
 array([-0.59479206, -0.14751631,  0.04470062,  0.04020178, -0.08343376,
        -0.10029656])]

### Phase II

In [12]:
partition_percentage = 0.5
partition_size = int(n_layer_steps*n_layers_to_add*partition_percentage)
n_partition_weights = partition_size*n_qubits
n_sweeps = 2

In [17]:
print(layer_weights[:5])
print(layer_gates[:5])

[array([ 2.10400159e-01,  1.66533454e-17, -5.47083698e-01,  2.63677968e-17,
        5.68989300e-17,  1.80411242e-17]), array([ 0.43871574,  0.72450691, -0.5470837 , -0.80994176,  0.45738643,
        1.15241499]), array([-0.45456927,  0.50659424,  1.24572831,  1.38817122,  0.30235989,
        0.5777338 ]), array([ 0.04519363,  0.02070302,  0.50438761, -0.23495578, -0.04768922,
       -0.10360508]), array([-0.46459097,  0.63507508, -0.1146228 , -0.30094421,  0.93010568,
        0.15200487])]
[[<class 'pennylane.ops.qubit.RX'>, <class 'pennylane.ops.qubit.RZ'>, <class 'pennylane.ops.qubit.RX'>, <class 'pennylane.ops.qubit.RZ'>, <class 'pennylane.ops.qubit.RZ'>, <class 'pennylane.ops.qubit.RZ'>], [<class 'pennylane.ops.qubit.RY'>, <class 'pennylane.ops.qubit.RY'>, <class 'pennylane.ops.qubit.RX'>, <class 'pennylane.ops.qubit.RX'>, <class 'pennylane.ops.qubit.RY'>, <class 'pennylane.ops.qubit.RY'>], [<class 'pennylane.ops.qubit.RX'>, <class 'pennylane.ops.qubit.RX'>, <class 'pennylane.ops.q

In [21]:
trainable_layers = layer_gates[:partition_size]
trainable_weights = layer_weights[:partition_size]
weights_flatten = np.concatenate(trainable_weights).ravel()

#is going to depend on the part of the circuit
@qml.template
def apply_partition(layer_gates, layer_weights)

tensor([ 2.10400159e-01,  1.66533454e-17, -5.47083698e-01,
         2.63677968e-17,  5.68989300e-17,  1.80411242e-17,
         4.38715740e-01,  7.24506914e-01, -5.47083698e-01,
        -8.09941763e-01,  4.57386427e-01,  1.15241499e+00,
        -4.54569267e-01,  5.06594237e-01,  1.24572831e+00,
         1.38817122e+00,  3.02359885e-01,  5.77733795e-01,
         4.51936345e-02,  2.07030219e-02,  5.04387608e-01,
        -2.34955782e-01, -4.76892210e-02, -1.03605083e-01,
        -4.64590972e-01,  6.35075082e-01, -1.14622804e-01,
        -3.00944214e-01,  9.30105678e-01,  1.52004868e-01], requires_grad=True)